In [1]:
!pip install ciw 
!pip install chart_studio
!pip3 install plotly

In [12]:
import ciw
from ciw.dists import *
import math
import json
from collections import defaultdict
import chart_studio.plotly as py
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from IPython.display import display, Markdown, HTML

import datetime
import zipfile

import pickle
import pandas as pd
import numpy as np
import random

import plotly
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
######################################################
# Constants

TICKS_IN_HOUR = 60
TICKS_IN_DAY = 24 * TICKS_IN_HOUR
TICKS_IN_YEAR = 365 * TICKS_IN_DAY



def build_run_fig(q, title="Run Data"):    
    
    fig = make_subplots(rows=4, cols=1, subplot_titles=("Service Time", "Arrival Queue", "Wait Time", "Latency"))
    df = pd.DataFrame(data={
            'timestamp' : [ rec.arrival_date for rec in q.get_all_records() ],
            'waittime' : [ rec.waiting_time for rec in q.get_all_records() ],
            'svctime' : [ rec.service_time for rec in q.get_all_records() ],
            'Latency' : [ rec.exit_date - rec.arrival_date for rec in q.get_all_records() ],
            'arr_q' : [ rec.queue_size_at_arrival for rec in q.get_all_records() ],
            'dep_q' : [ rec.queue_size_at_departure for rec in q.get_all_records() ]
        })
    df.sort_values('timestamp', inplace=True)
    fig.append_trace(go.Scatter(x=df['timestamp'], y=df['svctime']), row=1, col=1)
    fig.append_trace(go.Scatter(x=df['timestamp'], y=df['waittime']), row=2, col=1)
    fig.append_trace(go.Scatter(x=df['timestamp'], y=df['Latency']), row=3, col=1)
    fig.append_trace(go.Scatter(x=df['timestamp'], y=df['arr_q']), row=4, col=1)
    fig.update_layout(showlegend=False, height=800, width=800, title_text=title)
    return fig

def build_df_from_simulation(q,classes=None):
    if classes:
        columns=["queue", "timestamp","waittime","svctime","latency", "arr_q"]
    else:
        columns=["timestamp","waittime","svctime","latency", "arr_q"]
    
    df = pd.DataFrame(columns=columns)
    for ind, rec in enumerate(q.get_all_records()):
        if classes:
            df.loc[ind] ={
                    'queue' : classes[rec.customer_class],
                    'timestamp' : rec.arrival_date,
                    'waittime' : rec.waiting_time,
                    'svctime' : rec.service_time,
                    'latency' : rec.exit_date - rec.arrival_date,
                    'arr_q' : rec.queue_size_at_arrival
                }
        else:
            df.loc[ind] ={
                    'timestamp' : rec.arrival_date,
                    'waittime' : rec.waiting_time,
                    'svctime' : rec.service_time,
                    'latency' : rec.exit_date - rec.arrival_date,
                    'arr_q' : rec.queue_size_at_arrival
                }
        
    return df

def build_multi_bar(qlst, name_lst, column_name, title="Run Data", slo=None, classes=None):
    fig = go.Figure()
    max_y = 0
    
    for ind,q in enumerate(qlst):
        df = build_df_from_simulation(q, classes)
        tmp = np.histogram(df[column_name])[0].max()
        max_y = max_y if tmp < max_y else tmp
        
        if classes:
            fig.add_trace(go.Histogram(x=df[column_name], name=name_lst[ind], color="queue"))
        else:
            fig.add_trace(go.Histogram(x=df[column_name], name=name_lst[ind]))
        
    fig.update_layout(barmode='overlay')
    fig.update_traces(opacity=0.60)
    print("MAX:",max_y)
    if slo:
        
        fig.add_shape(dict(type="line",
                      x0=slo, y0=1, x1=slo, y1=max_y,
                      line=dict(color="Red",width=3)))
    return fig, df


def build_second_run_fig(q, title="Run Data"):

    fig_lst = []

    for cclass  in [ 0, 1, 2 ]:
        fig = make_subplots(rows=4, cols=1,
                            subplot_titles=("Service Time", "Arrival Queue", "Wait Time", "Latency"))

        df = pd.DataFrame(data={
                'timestamp' : [ rec.arrival_date for rec in q.get_all_records() if rec.customer_class == cclass ],
                'waittime' : [ rec.waiting_time for rec in q.get_all_records() if rec.customer_class == cclass ],
                'svctime' : [ rec.service_time for rec in q.get_all_records() if rec.customer_class == cclass],
                'Latency' : [ rec.exit_date - rec.arrival_date for rec in q.get_all_records() if rec.customer_class == cclass ],
                'arr_q' : [ rec.queue_size_at_arrival for rec in q.get_all_records() if rec.customer_class == cclass ],
            })

        df.sort_values('timestamp', inplace=True)

        fig.append_trace(go.Scatter(x=df['timestamp'], y=df['svctime']), row=1, col=1)
        fig.append_trace(go.Scatter(x=df['timestamp'], y=df['arr_q']), row=2, col=1)
        fig.append_trace(go.Scatter(x=df['timestamp'], y=df['waittime']), row=3, col=1)
        fig.append_trace(go.Scatter(x=df['timestamp'], y=df['Latency']), row=4, col=1)

        fig.update_layout(showlegend=False, height=800, width=800, title_text=title + " Class " + str(cclass))

        fig_lst.append(fig)

    return fig_lst

def build_second_run_bar(q, title="Run Data"):
    # Builds a bar graph based on what you hand in ..
    
    # build second has customer classes
    
    classes =  { 2 : 'LOW', 1 : 'MED', 0 : 'HIGH' }

    import plotly.express as px

    df = pd.DataFrame(columns=["queue", "timestamp","waittime","svctime","Latency", "arr_q"])

    for ind, rec in enumerate(q.get_all_records()):
        df.loc[ind] ={
                'queue' : classes[rec.customer_class],
                'timestamp' : rec.arrival_date,
                'waittime' : rec.waiting_time,
                'svctime' : rec.service_time,
                'Latency' : rec.exit_date - rec.arrival_date,
                'arr_q' : rec.queue_size_at_arrival
            }

    fig = px.histogram(df, x="waittime",color="queue",opacity=.4, histnorm='probability density')

    return fig, df

In [70]:
CUR_COLOR = '#2E7D32'
PREV_COLOR = '#A5D6A7'

def build_ts(values, start=pd.Timestamp.now(tz="America/New_York"), resample='5T'):
    tmp = pd.DatetimeIndex([ start + pd.Timedelta(datetime.timedelta(minutes=rec)) for rec in values ])
    arrival = pd.Series(np.ones(tmp.size),index=tmp)
    f = arrival.resample(resample).sum()
    return go.Scatter(x=f.index, y=f,
                      showlegend=False,
                      line=dict(color=CUR_COLOR,width=1)) 

def build_dashboard(q, prev_q=None, 
                    wait_time_slo=15, 
                    latency_slo=30,
                    title="Run Data"):

    #
    # Utilization [gauge]
    # Latency [bargraph]
    # 
    #
    
    # If we didn't get a prev_q var we'll plot latency 
    num_rows = 5
    sub_titles = ["Throughput","Utilization", "Latency Distribution", "Wait Time Distribution","Arrival Rate", "Latency"]
    specs = [ [{"type": "domain"},{"type": "domain"}],
                                   [  {"type": "xy","colspan":2}, None  ],                                   
                                   [  {"type": "xy","colspan":2}, None  ],
                                   [  {"type": "xy","colspan":2}, None  ],
                                   [  {"type": "xy","colspan":2}, None  ]
                                   ]
    if prev_q is not None:
        # We aren't goign to plot latency
        num_rows = 4
        sub_titles =["Throughput","Utilization", "Latency Distribution", "Wait Time Distribution","Arrival Rate"]
        specs = [ [{"type": "domain"},{"type": "domain"}],
                                   [  {"type": "xy","colspan":2}, None  ],                                   
                                   [  {"type": "xy","colspan":2}, None  ],
                                   [  {"type": "xy","colspan":2}, None  ],
                                   ]
    
    main_fig = make_subplots(rows=num_rows, cols=2,
#                            row_heights=[0.4, 0.3, 0.3],
                            row_titles=[None,"Latency"],
                            subplot_titles=sub_titles,
                            specs=specs)

    throughput = len(q.get_all_records()) / q.current_time
    utilization = [ node.server_utilisation for node in Q.nodes[1:-1] ][0]
    if prev_q != None:
        prev_throughput = len(prev_q.get_all_records()) / prev_q.current_time
        prev_utilization = [ node.server_utilisation for node in prev_q.nodes[1:-1] ][0]

        tp_indicator = go.Indicator(mode = "number+delta",
                                        value = throughput,
                                        number = {'suffix': "/sec"},
                                        delta = {'position':'bottom','reference': prev_throughput},
                                        domain = {'x': [0, 1], 'y': [0, 1]})
        ut_indicator = go.Indicator(domain = {'x': [0, 1], 'y': [0, 1]},
                                    value = utilization,
                                     mode = "gauge+number+delta",
                                     delta = {'reference' : prev_utilization},
                                     number = {'suffix' : '%'},
                                     gauge = {'axis': {'range': [None, 100]},
                                              'steps' : [ {'range': [0, 70], 'color': "lightgray"},
                                                          {'range': [70, 90], 'color': "gray"}],
                                              'threshold' : { 'line': {'color': "red", 'width': 4}, 
                                                              'thickness': 0.75, 'value': 90}})
    else:
        tp_indicator = go.Indicator(mode = "number",
                                        value = throughput,
                                        number = {'suffix': "/sec"},
                                        domain = {'x': [0, 1], 'y': [0, 1]})
        ut_indicator = go.Indicator(domain = {'x': [0, 1], 'y': [0, 1]},
                                    value = utilization,
                                     mode = "gauge+number",
                                     number = {'suffix' : '%'},
                                     gauge = {'axis': {'range': [None, 100]},
                                              'steps' : [ {'range': [0, 70], 'color': "lightgray"},
                                                          {'range': [70, 90], 'color': "gray"}],
                                              'threshold' : { 'line': {'color': "red", 'width': 4}, 
                                                              'thickness': 0.75, 'value': 90}})

    main_fig.add_trace(tp_indicator,1,1)
    main_fig.add_trace(ut_indicator,1,2)

    #
    # Latency Distribution
    #
    late_data = [ rec.exit_date - rec.arrival_date for rec in q.get_all_records() if rec.exit_date > rec.arrival_date ]
    top_y = len(q.get_all_records())

    main_fig.add_trace(go.Histogram(x=late_data, name="Current", marker_color=CUR_COLOR,bingroup=1),
                       row=2,col=1)

    if prev_q != None:
        prev_late_data = [ rec.exit_date - rec.arrival_date for rec in prev_q.get_all_records() if rec.exit_date > rec.arrival_date ]

        main_fig.add_trace(go.Histogram(x=prev_late_data, name="Prev", marker_color=PREV_COLOR,bingroup=1),
                           row=2,col=1)
        
    # SLO Line / Text
    main_fig.add_trace(go.Scatter(x=[latency_slo,latency_slo], y=[0,top_y],
                                    mode="lines+text",
                                    text=[None, "SLO"],
                                    showlegend=False,
                                    textposition="bottom center",
                                    line=dict(color='red',width=3)), 
                                    row=2, col=1)

    #
    # Wait time distribution
    #
    wait_data = [ rec.waiting_time for rec in q.get_all_records() if rec.waiting_time > 0]

    main_fig.add_trace(go.Histogram(x=wait_data, marker_color=CUR_COLOR,
                                    showlegend=False,bingroup=2),
                       row=3,col=1)

    # main_fig.add_shape(dict(type="line",
    #                         x0=wait_time_slo, y0=1, x1=wait_time_slo, y1=top_y,
    #                         line=dict(color="Red",width=3)),row=3,col=1)


    if prev_q != None:
        prev_wait_data = [ rec.waiting_time for rec in prev_q.get_all_records() if rec.waiting_time > 0 ]

        main_fig.add_trace(go.Histogram(x=prev_wait_data, showlegend=False, marker_color=PREV_COLOR,
                                        bingroup=2),
                          row=3,col=1)

    # SLO Line + Text
    main_fig.add_trace(go.Scatter(x=[wait_time_slo,wait_time_slo], y=[0,top_y],
                                    mode="lines+text",
                                    name="SLO",
                                    text=[None, "SLO"],
                                    textposition="bottom center",
                                    line=dict(color='red',width=3)), 
                                    row=3, col=1)

    main_fig.update_layout(barmode="overlay", bargap=0.1, title={
        'text':  title,
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'font': {'size': 36},
        'yanchor': 'top'})

    #
    # Arrival Timeseries
    #
    ts = build_ts([ rec.arrival_date for rec in q.get_all_records() ])

    main_fig.add_trace(ts, row=4, col=1)
    
    if prev_q is None:
        lat_times = [ rec.service_time for rec in q.get_all_records() ]
        
        main_fig.add_trace(go.Scatter(x=np.arange(len(lat_times)), y=lat_times,
                      showlegend=False,
                      line=dict(color=CUR_COLOR,width=1)), row=5, col=1)
        #ts = build_ts([ rec.service_time for rec in q.get_all_records() ])
        #main_fig.add_trace(ts, row=5, col=1)

    return main_fig

# All your queues belong to us: Optimizing Human in the Loop
* Matt Peters 
* Peter Silberman
![title foo](img/expel_logo.png "Title")

<h1>Introduction</h1>

What we're going to cover in this talk:

<ul>
    <li>General Idea of Queues / Queueing</li>
    <li>CIW Simulation Framework</li>
    <li>Application to Security Operations</li>
</ul>

<h1>Queues Explained - Parameters</h1>

![caption](img/queue-intro.png)

There are several parameters that define a queueing system
<ul>
    <li><b>Arrival Rate</b> - &lambda; - How fast does work arrive?</li>
    <li><b>Service Rate</b> - &mu; - How long does each job take to do?</li>
    <li><b>Number of servers</b> - How many jobs can we do at the same time</li>
    <li><b>Classes of service</b> - what order to we process work in?</li>
</ul>

<h1>Queues Explained - Observation</h1>

![caption](img/arrival-times.png)    
    
If we're in the middle, then we can try to optimize for:

<ul>
    <li><b>Throughput</b> - How many jobs per unit time can we do?</li>
    <li><b>Latency</b> - What's the end-to-end time for a job to travel through the system?</li>
    <li><b>Utilization</b> - How busy is each server in our system?</li>
</ul>



<h1>Queues In The Real World</h1>

![caption](img/queue-stock.jpg)

<h1>Simulating Queues - A Simple Example</h1>

<img src="img/queue-demo-1-new.png" alt="drawing"/>


In [8]:
network = ciw.create_network(arrival_distributions=[ Deterministic(4) ], 
                             service_distributions=[ Deterministic(5) ], 
                             number_of_servers=[1]) 

Q = ciw.Simulation(network)
Q.simulate_until_max_time(1440)

In [71]:
# Display The simulation Results Via Dashboard:
dashboard = build_dashboard(Q, title='A Simple Example')
dashboard.show()

<h1>What Happens if arrival or service aren't constant?</h1>

<img src="img/queue-random-functions-new.png" alt="drawing"/>



In [15]:
# Note: We changed our distributions below -- they are now exponentially distributed around
#       means of 4 and 5 min, respectively.
network = ciw.create_network(arrival_distributions=[ ciw.dists.Exponential(1.0/4) ],
                             service_distributions=[ ciw.dists.Exponential(1.0/4) ],
                             number_of_servers=[1])
Q1 = ciw.Simulation(network)
Q1.simulate_until_max_time(1440)

network = ciw.create_network(arrival_distributions=[ ciw.dists.Exponential(1.0/4) ],
                             service_distributions=[ ciw.dists.Exponential(1.0/6) ],
                             number_of_servers=[1])
Q2 = ciw.Simulation(network)
Q2.simulate_until_max_time(1440)

In [72]:
# Display The simulation Results Via Dashboard:
dashboard = build_dashboard(Q1, prev_q=Q2, title='Variable Service Times')
dashboard.show()

<h1>So What? How does this apply to me?</h1>

<img src="img/soc-as-queues.png" alt="drawing" width="500" height="500"/>

<b>Your SOC/SRE/Dev team is a queueing system. Understanding the parameters allows you to tune the output</b>

<ul>
    <li><b>Utilization</b> - This is your team, which usually feels like they're at 110%</li>
    <li><b>Throughput</b> - How many things can you triage per unit time</li>
    <li><b>Latency</b> - This is your SLO/SLA - how long until you see something?</li>
</ul>



<h1>Our Example SOC</h1>

<img src="img/our-soc-new.png" alt="drawing" width="700"/>

    with zipfile.ZipFile("arr_times_csv.zip").open('arr_times.csv') as fd:
        for line in fd.readlines()[:3]:
            print(line)

In [53]:
with zipfile.ZipFile("arr_times_csv.zip").open('arr_times.csv') as fd:
    for line in fd.readlines()[:3]:
        print(line)

b'Month Of,Created At\n'
b'2019-07,2019-07-19 00:00:14+00:00\n'
b'2019-07,2019-07-19 00:02:32+00:00\n'


# Historical Distro Slide
    class HistoricalDistribution(ciw.dists.Distribution):
        def __init__(self, sev='TOTAL',
                           filename='arr_times.pickle', 
                           adjustment_factor=0.0):
                           
            self.dists = defaultdict(dict)

            counts = get_arr_counts(filename, sev=sev)
            for day in counts.keys():
                for hr, avg_per_min in counts[day].items():
                    self.dists[day][hr] = ciw.dists.Exponential(avg_per_min - \
                                                (avg_per_min * adjustment_factor))

        def sample(self, t, ind=None):
            day    = math.floor((t / TICKS_IN_DAY) % 7)
            hour   = math.floor((t / TICKS_IN_HOUR) % 24)
            sample = self.dists[day][hour].sample(t,ind)
            return sample



# Service Distro Slide
    class ServiceDistribution(ciw.dists.Distribution):
        def __init__(self, avg_time_to_triage, 
                           avg_time_to_investigate,
                           avg_time_to_report, 
                           prob_of_inv=.2):
            self.time_to_triage      = Exponential(1/avg_time_to_triage)
            self.time_to_investigate = Exponential(1/avg_time_to_investigate)
            self.time_to_report      = Exponential(1/avg_time_to_report)
            self.prob_of_inv         = prob_of_inv

        def sample(self, t, ind=None):
            total_time = self.time_to_triage.sample()
            if random.random() < self.prob_of_inv:
                total_time += self.time_to_investigate.sample()
                total_time += self.time_to_report.sample()

            return total_time   
            

    TIME_TO_TRIAGE      = 4
    TIME_TO_INVESTIGATE = 20
    TIME_TO_REPORT      = 10
    NUM_SOC_WORKERS     = 1
    SIMULATION_TIME     = 24*60


    network = ciw.create_network(arrival_distributions=[ HistoricalDistribution() ],
                                 service_distributions=[ 
                                     ServiceDistribution(TIME_TO_TRIAGE,
                                                         TIME_TO_INVESTIGATE,
                                                         TIME_TO_REPORT) 
                                 ],
                                 number_of_servers=[ NUM_SOC_WORKERS ])
    Q = ciw.Simulation(network)
    Q.simulate_until_max_time(SIMULATION_TIME)

In [22]:
def get_arr_counts(filename, sev='TOTAL'):
    with open(filename, 'rb') as fd:
        dd = pickle.load(fd)

    counts = {day: {hour : 0.0 for hour in range(24)} for day in range(7)}    
    for (day, hr_sevs) in dd.items():
        for hr in hr_sevs.keys():
            cnts = dd[day][hr][sev]
            counts[day][hr] =  ((sum(cnts) / 3120)) # 52*60
    return counts


class HistoricalDistribution(ciw.dists.Distribution):

    def __init__(self, sev='TOTAL', filename='arr_times.pickle', adjustment_factor=0.0):
        self.dists = defaultdict(dict)
        
        counts = get_arr_counts(filename, sev=sev)
        for day in counts.keys():
            for hr, avg_per_min in counts[day].items():
                self.dists[day][hr] = ciw.dists.Exponential(avg_per_min -(avg_per_min * adjustment_factor) )
        
    def sample(self, t, ind=None):
        day    = math.floor((t / TICKS_IN_DAY) % 7)
        hour   = math.floor((t / TICKS_IN_HOUR) % 24)
        sample = self.dists[day][hour].sample(t,ind)
        return sample

class ServiceDistribution(ciw.dists.Distribution):
    def __init__(self, avg_time_to_triage, avg_time_to_investigate,
                 avg_time_to_report, prob_of_inv=.2):
        self.time_to_triage      = Exponential(1/avg_time_to_triage)
        self.time_to_investigate = Exponential(1/avg_time_to_investigate)
        self.time_to_report      = Exponential(1/avg_time_to_report)
        self.prob_of_inv         = prob_of_inv

    def sample(self, t, ind=None):
        total_time = self.time_to_triage.sample()
        if random.random() < self.prob_of_inv:
            total_time += self.time_to_investigate.sample()
            total_time += self.time_to_report.sample()

        return total_time            

TIME_TO_TRIAGE      = 4
TIME_TO_INVESTIGATE = 20
TIME_TO_REPORT      = 10
NUM_SOC_WORKERS     = 1
SIMULATION_TIME     = 24*60


network = ciw.create_network(arrival_distributions=[ HistoricalDistribution() ],
                             service_distributions=[ 
                                     ServiceDistribution(
                                                     TIME_TO_TRIAGE,
                                                     TIME_TO_INVESTIGATE,
                                                     TIME_TO_REPORT) 
                                     ],
                             number_of_servers=[ NUM_SOC_WORKERS ])

Q_basic_soc = ciw.Simulation(network)
Q_basic_soc.simulate_until_max_time(SIMULATION_TIME)

In [73]:
dashboard = build_dashboard(Q_basic_soc, title='Basic SOC')
dashboard.show()

<h1>Our Example SOC - Adding Capacity</h1>

<img src="img/our-soc-capacity.png" alt="drawing" width="600"/>

Here we add staff, so the number of servers (SOC workers) increases.

    NUM_SOC_WORKERS = 2

    network = ciw.create_network(arrival_distributions=[ HistoricalDistribution() ],
                                 service_distributions=[ 
                                     ServiceDistribution(TIME_TO_TRIAGE,
                                                         TIME_TO_INVESTIGATE,
                                                         TIME_TO_REPORT) 
                                 ],
                                 number_of_servers=[ NUM_SOC_WORKERS ])

    Q = ciw.Simulation(network)

    Q.simulate_until_max_time(SIMULATION_TIME)

In [26]:
NUM_SOC_WORKERS = 2

network = ciw.create_network(arrival_distributions=[ HistoricalDistribution() ],
                             service_distributions=[ ServiceDistribution(TIME_TO_TRIAGE,
                                                                         TIME_TO_INVESTIGATE,
                                                                         TIME_TO_REPORT) ],
                             number_of_servers=[ NUM_SOC_WORKERS ])

Q_add_cap = ciw.Simulation(network)

Q_add_cap.simulate_until_max_time(SIMULATION_TIME)

In [74]:
dashboard = build_dashboard(Q_add_cap, prev_q=Q_basic_soc, title='Added Capacity vs Basic SOC')
dashboard.show()

<h1>Our Example SOC - Training</h1>

<img src="img/our-soc-training.png" alt="drawing" width="700"/>

Here we add training, so the probability of investigation is reduced by 10%, as well as the time it takes by 20%.

    TIME_TO_INVESTIGATE  *= .20
    TIME_TO_REPORT       *= .20
    PROB_OF_INVESTIGATION = .18

    network = ciw.create_network(arrival_distributions=[ HistoricalDistribution() ],
                                 service_distributions=[ 
                                         ServiceDistribution(TIME_TO_TRIAGE,
                                                             TIME_TO_INVESTIGATE,
                                                             TIME_TO_REPORT,
                                                             PROB_OF_INVESTIGATION) 
                                                  ],
                                number_of_servers=[ NUM_SOC_WORKERS ])

    Q = ciw.Simulation(network)

    Q.simulate_until_max_time(simulation_time)

In [55]:
TIME_TO_INVESTIGATE  *= .20
TIME_TO_REPORT       *= .20
PROB_OF_INVESTIGATION = .18

network = ciw.create_network(arrival_distributions=[ HistoricalDistribution() ],
                             service_distributions=[ 
                                             ServiceDistribution(TIME_TO_TRIAGE,
                                                                 TIME_TO_INVESTIGATE,
                                                                 TIME_TO_REPORT,
                                                                 PROB_OF_INVESTIGATION) ],
                            number_of_servers=[ NUM_SOC_WORKERS ])

Q_train = ciw.Simulation(network)
Q_train.simulate_until_max_time(SIMULATION_TIME)

In [75]:
dashboard = build_dashboard(Q_train, prev_q=Q_add_cap, title='Added Capacity vs Training')
dashboard.show()

<h1>Our Example SOC - Tuning</h1>

<img src="img/our-soc-tuning.png" alt="drawing" width="700"/>

Here we add tuning so the number of alerts arriving in our SOC is reduced by 15%.

    ADJUSTMENT_FACTOR = .85
    PROB_OF_INVESTIGATION  = 0.2 * .1
    network = ciw.create_network(arrival_distributions=[ 
                                    HistoricalDistribution(
                                                adjustment_factor=ADJUSTMENT_FACTOR) 
                                 ],
                                 service_distributions=[ 
                                     ServiceDistribution(TIME_TO_TRIAGE,
                                                         TIME_TO_INVESTIGATE,
                                                         TIME_TO_REPORT,
                                                         PROB_OF_INVESTIGATION) 
                                 ],
                                 number_of_servers=[ NUM_SOC_WORKERS ])

    Q = ciw.Simulation(network)

    Q.simulate_until_max_time(SIMULATION_TIME)

In [30]:
ADJUSTMENT_FACTOR = .15
PROB_OF_INVESTIGATION  = 0.2 * .1
network = ciw.create_network(arrival_distributions=[ HistoricalDistribution(adjustment_factor=ADJUSTMENT_FACTOR) ],
                             service_distributions=[ ServiceDistribution(TIME_TO_TRIAGE,
                                                                         TIME_TO_INVESTIGATE,
                                                                         TIME_TO_REPORT,
                                                                         PROB_OF_INVESTIGATION) ],

                             number_of_servers=[ NUM_SOC_WORKERS ])

Q_tuning = ciw.Simulation(network)
Q_tuning.simulate_until_max_time(SIMULATION_TIME)

In [31]:
dashboard = build_dashboard(Q_tuning, prev_q=Q_train)
dashboard.show()

<h1>Our Example SOC - Service Classes</h1>

<img src="img/our-soc-classes.png" alt="drawing" width="700"/>

Here we add service classes, so our alerts are now associated with a severity (HIGH, MED, LOW) which are each handled in order.

    priority_classes={ 'Class 0' : 0,
                       'Class 1' : 1,
                       'Class 2' : 2 }

    arrival_distributions = { 'Class 0' :  [ HistoricalDistribution(sev='HIGH', 
                                                    adjustment_factor=ADJUSTMENT_FACTOR) ],
                              'Class 1' :  [ HistoricalDistribution(sev='MEDIUM', 
                                                    adjustment_factor=ADJUSTMENT_FACTOR) ],
                              'Class 2' :  [ HistoricalDistribution(sev='LOW', 
                                                    adjustment_factor=ADJUSTMENT_FACTOR) ] }


    service_distributions={ 'Class 0' : [ ServiceDistribution(TIME_TO_TRIAGE,
                                                                TIME_TO_INVESTIGATE,
                                                                TIME_TO_REPORT,
                                                                PROB_OF_INVESTIGATION) ],
                            'Class 1' : ServiceDistribution(TIME_TO_TRIAGE,
                                                                TIME_TO_INVESTIGATE,
                                                                TIME_TO_REPORT,
                                                                PROB_OF_INVESTIGATION) ],
                            'Class 2' : ServiceDistribution(TIME_TO_TRIAGE,
                                                                TIME_TO_INVESTIGATE,
                                                                TIME_TO_REPORT,
                                                                PROB_OF_INVESTIGATION) ] }

    network = ciw.create_network(arrival_distributions=arrival_distributions,
                                 service_distributions=service_distributions,
                                 priority_classes=priority_classes,
                                 number_of_servers=[ NUM_SOC_WORKERS ])

    Q = ciw.Simulation(network)
    Q.simulate_until_max_time(SIMULATION_TIME)

In [32]:
priority_classes={ 'Class 0' : 0,
                   'Class 1' : 1,
                   'Class 2' : 2 }

arrival_distributions = { 'Class 0' :  [ HistoricalDistribution(sev='HIGH', adjustment_factor=ADJUSTMENT_FACTOR) ],
                          'Class 1' :  [ HistoricalDistribution(sev='MEDIUM', adjustment_factor=ADJUSTMENT_FACTOR) ],
                          'Class 2' :  [ HistoricalDistribution(sev='LOW', adjustment_factor=ADJUSTMENT_FACTOR) ] }


service_distributions={ 'Class 0' : [ ServiceDistribution(TIME_TO_TRIAGE, TIME_TO_INVESTIGATE,
                                                          TIME_TO_REPORT, PROB_OF_INVESTIGATION) ],
                        'Class 1' : [ ServiceDistribution(TIME_TO_TRIAGE, TIME_TO_INVESTIGATE,
                                                          TIME_TO_REPORT, PROB_OF_INVESTIGATION) ],
                        'Class 2' : [ ServiceDistribution(TIME_TO_TRIAGE, TIME_TO_INVESTIGATE,
                                                          TIME_TO_REPORT, PROB_OF_INVESTIGATION) ] }

network = ciw.create_network(arrival_distributions=arrival_distributions,
                             service_distributions=service_distributions,
                             priority_classes=priority_classes,
                             number_of_servers=[ NUM_SOC_WORKERS ])

Q_svc_cls = ciw.Simulation(network)
Q_svc_cls.simulate_until_max_time(SIMULATION_TIME)

In [33]:
dashboard = build_dashboard(Q_svc_cls, prev_q=Q_tuning)
dashboard.show()

<h1>Our Example SOC - Tiering</h1>

<img src="img/our-soc-network.png" alt="drawing" width="800"/>

Here we add tiering to our SOC -- we'll have different analysts investigate alerts after we triage them.

    class InvestigationDistribution(ciw.dists.Distribution):
        def __init__(self, avg_time_to_investigate, avg_time_to_report):
            self.time_to_investigate = ciw.dists.Exponential(1/avg_time_to_investigate)
            self.time_to_report = ciw.dists.Exponential(1/avg_time_to_report)

        def sample(self, t, ind=None):
            return self.time_to_investigate.sample() + self.time_to_report.sample()

    number_of_servers = [ 2, 1, 1 ]
    
    arrival_per_class = { 'Class 0' : [HistoricalDistribution(sev='HIGH', 
                                                      adjustment_factor=ADJUSTMENT_FACTOR), 
                                                    NoArrivals(), 
                                                    NoArrivals() ],
                          'Class 1' : [HistoricalDistribution(sev='MEDIUM', 
                                                      adjustment_factor=ADJUSTMENT_FACTOR), 
                                                     NoArrivals(), 
                                                     NoArrivals() ],
                          'Class 2' : [HistoricalDistribution(sev='LOW', 
                                                      adjustment_factor=ADJUSTMENT_FACTOR), 
                                                     NoArrivals(), 
                                                     NoArrivals() ] }

    service_distributions = [ Exponential(1/TIME_TO_TRIAGE), 
                              InvestigationDistribution(TIME_TO_INVESTIGATE,
                                                        TIME_TO_REPORT), 
                              Deterministic(0) ]

    service_per_class = { 'Class 0' : service_distributions,
                          'Class 1' : service_distributions,
                          'Class 2' : service_distributions }

    std_routing = [ [0.0, PROB_OF_INVESTIGATION, 1-PROB_OF_INVESTIGATION ],
                    [0.0, 0.0, 1.0],
                    [0.0, 0.0, 0.0] ]

    routing_matrix = { 'Class 0' : std_routing, 
                       'Class 1' : std_routing,
                       'Class 2' : std_routing }

    network = ciw.create_network(arrival_distributions=arrival_per_class,
                                 service_distributions=service_per_class,
                                 routing=routing_matrix,
                                 priority_classes=priority_classes,
                                 number_of_servers=number_of_servers)

    Q = ciw.Simulation(network)

    Q.simulate_until_max_time(SIMULATION_TIME)

In [35]:
class InvestigationDistribution(ciw.dists.Distribution):
    def __init__(self, avg_time_to_investigate, avg_time_to_report):
        self.time_to_investigate = ciw.dists.Exponential(1/avg_time_to_investigate)
        self.time_to_report = ciw.dists.Exponential(1/avg_time_to_report)

    def sample(self, t, ind=None):
        return self.time_to_investigate.sample() + self.time_to_report.sample()

number_of_servers = [ 2, 1, 1 ]

#arrival_distributions = [ HistoricalDistribution(ADJUSTMENT_FACTOR), NoArrivals(), NoArrivals() ]

arrival_per_class = { 'Class 0' : [HistoricalDistribution(sev='HIGH', adjustment_factor=ADJUSTMENT_FACTOR), NoArrivals(), NoArrivals() ],
                      'Class 1' : [HistoricalDistribution(sev='MEDIUM', adjustment_factor=ADJUSTMENT_FACTOR), NoArrivals(), NoArrivals() ],
                      'Class 2' : [HistoricalDistribution(sev='LOW', adjustment_factor=ADJUSTMENT_FACTOR), NoArrivals(), NoArrivals() ] }

service_distributions = [ Exponential(1/TIME_TO_TRIAGE), 
                          InvestigationDistribution(TIME_TO_INVESTIGATE,TIME_TO_REPORT), 
                          Deterministic(0) ]

service_per_class = { 'Class 0' : service_distributions,
                      'Class 1' : service_distributions,
                      'Class 2' : service_distributions }

std_routing = [ [0.0, PROB_OF_INVESTIGATION, 1-PROB_OF_INVESTIGATION ],
                [0.0, 0.0, 1.0],
                [0.0, 0.0, 0.0] ]

routing_matrix = { 'Class 0' : std_routing, 
                   'Class 1' : std_routing,
                   'Class 2' : std_routing }

network = ciw.create_network(arrival_distributions=arrival_per_class,
                             service_distributions=service_per_class,
                             routing=routing_matrix,
                             priority_classes=priority_classes,
                             number_of_servers=number_of_servers)

Q_tier = ciw.Simulation(network)
Q_tier.simulate_until_max_time(SIMULATION_TIME)

In [36]:
dashboard = build_dashboard(Q_tier, prev_q=Q_svc_cls)
dashboard.show()

<h1>Our Example SOC - Automation</h1>

<img src="img/our-soc-automated.png" alt="drawing" width="800"/>

Here we add some automation to the investigation phase -- we'll have some robots that can the reporting and some of the investigations.

    class AutoInvestigationDistribution(ciw.dists.Distribution):

        def __init__(self, avg_time_to_investigate, avg_time_to_report,
                     prob_of_automation=0):
            self.time_to_investigate = ciw.dists.Exponential(1/avg_time_to_investigate)
            self.time_to_report      = ciw.dists.Exponential(1/avg_time_to_report)
            self.prob_of_automation  = prob_of_automation

        def is_automated(self):
            return random.random() >= self.prob_of_automation

        def sample(self, t, ind=None):
            if self.is_automated():
                return 0
            return self.time_to_investigate.sample() + self.time_to_report.sample()

    PROB_OF_AUTOMATION = 0.4

    service_distributions = [ Exponential(1/TIME_TO_TRIAGE), 
                              AutoInvestigationDistribution(TIME_TO_INVESTIGATE,
                                                            TIME_TO_REPORT,
                                                            PROB_OF_AUTOMATION), 
                              Deterministic(0) ]


    service_per_class = { 'Class 0' : service_distributions,
                          'Class 1' : service_distributions,
                          'Class 2' : service_distributions }

    network = ciw.create_network(arrival_distributions=arrival_per_class,
                                 service_distributions=service_per_class,
                                 routing=routing_matrix,
                                 priority_classes=priority_classes,
                                 number_of_servers=number_of_servers)


    Q = ciw.Simulation(network)

    Q.simulate_until_max_time(SIMULATION_TIME)


In [57]:
class AutoInvestigationDistribution(ciw.dists.Distribution):

    def __init__(self, avg_time_to_investigate, avg_time_to_report,
                 prob_of_automation=0):
        self.time_to_investigate = ciw.dists.Exponential(1/avg_time_to_investigate)
        self.time_to_report      = ciw.dists.Exponential(1/avg_time_to_report)
        self.prob_of_automation  = prob_of_automation

    def is_automated(self):
        return random.random() >= self.prob_of_automation

    def sample(self, t, ind=None):
        if self.is_automated():
            return 0
        return self.time_to_investigate.sample() + self.time_to_report.sample()

PROB_OF_AUTOMATION = 0.4

service_distributions = [ Exponential(1/TIME_TO_TRIAGE), 
                          AutoInvestigationDistribution(TIME_TO_INVESTIGATE,
                                                                    TIME_TO_REPORT,
                                                                    PROB_OF_AUTOMATION), 
                          Deterministic(0) ]


service_per_class = { 'Class 0' : service_distributions,
                      'Class 1' : service_distributions,
                      'Class 2' : service_distributions }

network = ciw.create_network(arrival_distributions=arrival_per_class,
                             service_distributions=service_per_class,
                             routing=routing_matrix,
                             priority_classes=priority_classes,
                             number_of_servers=number_of_servers)

Q_auto = ciw.Simulation(network)
Q_auto.simulate_until_max_time(SIMULATION_TIME)

In [38]:
dashboard = build_dashboard(Q_auto, prev_q=Q_tier)
dashboard.show()

<h1>Conclusion</h1>
<img src="img/themoreyouknow.png" alt="drawing" width="800"/>
<ul>
    <li>Understand what you're optimizing for</li>
    <li>Change one thing at a time to observe effects</li>
    <li>Use this to guide real-world modifications of your process</li>
</ul>
